# Notebook for sampling SMPL-X poses and shapes

This notebook is meant to be run on Google colab, otherwise paths to subject data and modules might need adjustments

In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
### Download data if not done yet

import os
import zipfile
import urllib.request as request

attributes = ['body']

subjects = [1, 2, 3]

for subject in subjects:
    if not os.path.exists('subject_%d' % subject):
        for attr in attributes:
            url = os.path.join('https://humbi-dataset.s3.amazonaws.com', attr + '_subject', 'subject_%d.zip' % subject)
            path = '%s_subject_%d.zip' % (attr, subject)
            request.urlretrieve(url, path)
            downloaded_zip = zipfile.ZipFile(path)
            downloaded_zip.extractall() # !unzip downloaded_zip
            os.remove(path)

In [ ]:
import smplx
import random

from pytorch3d.io import save_obj
from pytorch3d.structures import Meshes
from pytorch3d.transforms import axis_angle_to_matrix

import sys
sys.path.append('utils')
from utils.smpl_to_smplx import extract_smpl_param
from utils.plot_structures import plot_structure

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
### Sampling functions
def sample_pose(body_poses_list):
    body_pose = random.choice(body_poses_smplx)
    return body_pose

def sample_shape(bedas_std=torch.ones([10])):
    betas_mean = torch.zeros([10])
    betas = torch.normal(betas_mean, bedas_std).unsqueeze(0)
    return betas

In [ ]:
### Get all poses from downloaded subjects
smplx_model_path = drive_path + 'smplx'
smplx_model = smplx.SMPLXLayer(smplx_model_path, gender='neutral')

save_path = drive_path + 'tmp_files/figures'

poses = []
body_poses_smplx = []

for subject in subjects:
    poses_path = 'subject_%d/body/' % subject
    for pose in sorted(os.listdir(poses_path)):
        pose_path = os.path.join(poses_path, pose)
        if os.path.isdir(pose_path):
            poses.append(pose)

for pose in poses:
    body_pose_smplx = extract_smpl_param(subject, pose)[-1]
    body_poses_smplx.append(body_pose_smplx)

body_pose = body_poses_smplx[0] # set to first pose
betas = torch.zeros([10]) # set to mean shape

In [ ]:
### Sample and plot
body_pose = sample_pose(body_poses_smplx) # comment if you want to keep the same pose
betas = sample_shape() # comment if you want to keep the same shape

smplx_faces = torch.Tensor(smplx_model.faces.astype('int')).type(torch.int32).unsqueeze(0)
smplx_verts = smplx_model.forward(body_pose=axis_angle_to_matrix(body_pose), betas=betas)['vertices']

smplx_mesh = Meshes(smplx_verts, smplx_faces)

plot_structure([smplx_mesh])

In [ ]:
### Store current SMPL-X pose and shape in mesh as .obj file
os.makedirs(save_path, exist_ok=True)
smplx_filename = os.path.join(save_path, 'sampled_smplx.obj')

save_obj(smplx_filename, verts=smplx_mesh.verts_packed(), faces=smplx_mesh.faces_packed())